In [8]:
import cv2
import numpy as np
from moviepy.editor import VideoFileClip, ImageSequenceClip
from google.colab.patches import cv2_imshow

In [9]:
def make_coordinates(image, line_parameters):
    slope, intercept = line_parameters
    y1 = image.shape[0]
    y2 = int(y1*(3/5))
    x1 = int((y1 - intercept)/slope)
    x2 = int((y2 - intercept)/slope)
    return np.array([x1, y1, x2, y2])

In [10]:
def average_slope_intercept(image, lines):
    left_fit = []
    right_fit = []
    for line in lines:
        x1, y1, x2, y2 = line.reshape(4)
        parameters = np.polyfit((x1, x2), (y1, y2), 1)
        slope = parameters[0]
        intercept = parameters[1]
        if slope < 0:
            left_fit.append((slope, intercept))
        else:
            right_fit.append((slope, intercept))
    leftfit_average = np.average(left_fit, axis=0)
    rightfit_average = np.average(right_fit, axis=0)
    left_line = make_coordinates(image, leftfit_average)
    right_line = make_coordinates(image, rightfit_average)
    return np.array([left_line, right_line])

In [11]:
def canny(image):
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    blur = cv2.GaussianBlur(gray, (5, 5), 0)
    canny_edges = cv2.Canny(blur, 50, 150)
    return canny_edges

In [12]:
def display_lines(img, lines):
    line_image = np.zeros_like(img)
    if lines is not None:
        for line in lines:
            x1, y1, x2, y2 = line.reshape(4)
            cv2.line(line_image, (x1,y1), (x2, y2), (255, 0, 0), 10)
    return line_image

In [13]:
def region_of_interest(img):
    height = img.shape[0]
    triangle = np.array([[(200, height), (1100, height), (550, 250)]])
    mask = np.zeros_like(img)
    cv2.fillPoly(mask, [triangle], 255)
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image

In [15]:
cap = cv2.VideoCapture("test2.mp4")
frames = []
if not cap.isOpened():
    print("Error opening video file")
    exit()
while(cap.isOpened()):
    ret, frame = cap.read()
    if not ret:
        break
    canny_edges = canny(frame)
    cropped_img = region_of_interest(canny_edges)
    lines = cv2.HoughLinesP(cropped_img, 2, np.pi/180, 100, np.array([]), minLineLength=40, maxLineGap=5)
    avg_lines = average_slope_intercept(frame, lines)
    line_image = display_lines(frame, avg_lines)
    combo_image =  cv2.addWeighted(frame, 0.8, line_image, 1, 1)
    frames.append(combo_image)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

clip = ImageSequenceClip(frames, fps=20)  # Create video clip
clip.write_videofile("output.mp4", codec="libx264")

Moviepy - Building video output.mp4.
Moviepy - Writing video output.mp4



Moviepy - Done !
Moviepy - video ready output.mp4
